# Export Data Exercise

In this exercise, you will use a python notebook to export data using the Scout SDK.

## Is this thing on?

Test to make sure your notebook is working. 

- Click the code cell below
- Click the play button to the left of the cell to execute its code
- You should see *testing...testing...is this thing on?* printed below the cell

In [ ]:
print('testing...testing...is this thing on?')

## Imports

Now we're going to import some modules we will use throughout this exercise. Execute the cell below to import the modules.

In [ ]:
#####################################
## RUN BUT DO NOT CHANGE THIS CELL ##
#####################################

import boto3
import time
import urllib3

from bosdyn.scout.client import ScoutClient

# We do not have an SSL Certificate, so ignore these warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# These variables will help connect to and authenticate with Scout
HOSTNAME = '10.202.101.3'
USERNAME = 'student'
PASSWORD = 'studentuser123'

# These variables will help us when getting captured data
MISSION = 'ThermalMissionExportData'
CHANNEL = 'thermal-inspection_advanced-anomaly_isotherm_image'
WALL = 'WallSideHeater'
ROOM = 'RoomSideHeater'

# These variables will help us when we write data to AWS
DAY = time.strftime('%Y%m%d')
BUCKET_NAME = 'export-data-training-bucket'

## Setting up the ScoutClient

Next we will set up our ScoutClient. We created variables for Scout's hostname, username, and password. We pass the username and password here to `authenticate_with_password()`. We did not have to. If we did not, we would have been prompted for a username and password. Additionally, we could have created environment variables `BOSDYN_SCOUT_CLIENT_USERNAME` and `BOSDYN_SCOUT_CLIENT_PASSWORD`.

After executing the following cell, you should see `10.202.101.3` printed. That is the IP address of our Scout instance.

In [ ]:
#####################################
## RUN BUT DO NOT CHANGE THIS CELL ##
#####################################

scout_client = ScoutClient(hostname=HOSTNAME, verify=False)
scout_client.authenticate_with_password(username=USERNAME, password=PASSWORD)

print('Scout IP:', scout_client._hostname)

## View the Robots on Scout

Here's an example of using the Scout SDK. We use our ScoutClient to call get_robots(). We get a requests response.

Then, we check that the response came back ok. If it did, we then get the JSON data from the response and loop through all the robots to print each hostname.

In [ ]:
#####################################
## RUN BUT DO NOT CHANGE THIS CELL ##
#####################################

get_robots_response = scout_client.get_robots()

if get_robots_response.ok:
    all_robots = get_robots_response.json()
    for robot in all_robots:
        robot_hostname = robot.pop('hostname', None)
        if robot_hostname:
            print(robot_hostname)
else:
    print('get_robots() failed')
    print(get_robots_response.text)

## View the Most Recent Runs

Let's view the most recent runs in Scout. A run is a period of mission playback or teleoperation. Use the get_runs() function to get the data.

The schema is:

```
{
    limit       integer
    offset      integer
    total       integer
    resources   [
                  Run {
                    ...
                  }
                ]
}
```

You can view more in the Scout Web API.

### Task 1

Print the most recent runs using get_runs().

Use the get_robots() example above to guide you.

In [ ]:
get_runs_response = # YOUR CODE HERE

if get_runs_response.ok:
    runs_json = # YOUR CODE HERE

    ############################################
    ## DO NOT CHANGE THE NEXT 3 LINES OF CODE ##
    ############################################
    runs = runs_json.get('resources') 
    for run in runs:
        print(run)

### Task 2

Print the most recent runs with the mission name 'ExportDataThermalMission'.

Use the params argument to get_runs() to filter for the mission name. Look at the Scout Web API to identify the correct parameter name.

In [ ]:
####################
## YOUR CODE HERE ##
####################




## View Action Data

Our goal is to look at the temperature of the reptile heaters during our missions.

`get_run_events()` and `get_run_captures()` will get us the thermal data.

`get_run_events()` returns more data, for our use case, the additional data is not necessary, so we will use `get_run_captures()`.

The schema for get_run_captures() is:

```
{
    limit       integer
    offset      integer
    total       integer
    resources   [
                  RunCapture {
                    ...
                    keyResults []
                    ...
                  }
                ]
}
```

Thermal data is stored in `keyResults`. A key result for our thermal data is:

```
{
    "roi": "Region 1",
    "name": "Max Temperature",
    "units": "K",
    "value": <float>
},
{
    "roi": "Region 1",
    "name": "Min Temperature",
    "units": "K",
    "value": <float>
}
```

We are interested in getting the minimum and maximum temperatures from the value fields.

### Task 3

Print the **time**, **Min Temperature**, **Max Temperature** for each thermal data capture.

Click this link to see the [run_captures data](https://training.lab.bosdyn.internal/api/v0/run_captures/).

There are two code blocks below. The first is partially filled out to help you get the data for the reptile heater closer to the wall. The second code block is empty and where you should place your code for the room side reptile heater. The code will be very similar to the wall side reptile heater.

In [ ]:
##########################################################
## USE THIS CODE BLOCK FOR THE WALL SIDE REPTILE HEATER ##
##########################################################

print(WALL)

wall_heater_params = {
    'missionName': 'ExportDataThermalMission',
    'actionName': 'WallReptileHeater',
    'channelName': 'thermal-inspection_advanced-anomaly_isotherm_image'
}
get_run_captures_response = # YOUR CODE HERE

if get_run_captures_response.ok:
    run_captures_json = # YOUR CODE HERE
    run_captures = # YOUR CODE HERE

    for capture in run_captures:
        time = capture.get('time')

        key_results = # YOUR CODE HERE
        max_temp = key_results[0].get('value')
        min_temp = key_results[1].get('value')
        
        print(f'{time}\t{max_temp}\t{min_temp}')

In [ ]:
##########################################################
## USE THIS CODE BLOCK FOR THE ROOM SIDE REPTILE HEATER ##
##########################################################

print('Room Side Reptile Heater')

####################
## YOUR CODE HERE ##
####################


## Export Data to AWS

Now that we can access the data that we need, we can export it to some external source. We will be exporting our data to an S3 bucket.

We will get the thermal data again for our two reptile heaters. This time, instead of just printing the data to the screen, we will write it to a file and then upload that file to S3.

You've already done the hard part, getting the data out of Scout. We wrote a couple of helper functions to get the thermal data and write it to a file on your computer.

`get_thermal_data()` does the work of getting the data from Scout and returning the thermal data. It does so for a specified mission and action.

`write_locally()` takes the data returned from `get_thermal_data()` and writes it to a CSV file.

In [ ]:
#####################################
## RUN BUT DO NOT CHANGE THIS CELL ##
#####################################

def get_thermal_data(action):
    data = []

    res = scout_client.get_run_captures(params={
        'missionName': MISSION,
        'actionName': action,
        'channelName': CHANNEL
    })

    if res.ok:
        captures = res.json().get('resources')
        for capture in captures:
            time = capture.get('time')
            key_results = capture.get('keyResults')
            max_temp = key_results[0].get('value')
            min_temp = key_results[1].get('value')
            data.append([time, max_temp, min_temp])
    return data


def write_locally(data, filename):
    with open(f'{filename}', 'w') as out:
        for line in data:
            out.write(f'{line[0]},{line[1]},{line[2]}\n')

### Task 4

- In the first code cell, complete the `upload_to_aws()` function. This function will be used twice, once for each reptile heater.

- In the final code cell, call the `upload_to_aws()` function twice, once for each reptile heater. When you upload to AWS, the filename should be unique. Make sure you fill out the `NAME` variable in the final code cell to ensure a unique filename.

- Use the variable `BUCKET_NAME` when specifying the S3 bucket.

In [ ]:
def upload_to_aws(source_filename, bucket_name, destination_filename):
    ####################
    ## YOUR CODE HERE ##
    ####################

In [ ]:
##########################################
## WRITE YOUR NAME IN THE NAME VARIABLE ##
##########################################
NAME = ''

# Get the thermal data and filenames
wall_data = get_thermal_data(WALL)
wall_filename = f'{DAY}-{WALL}-{NAME}.csv'
room_data = get_thermal_data(ROOM)
room_filename = f'{DAY}-{ROOM}-{NAME}.csv'

# Write the files locally
write_locally(wall_data, wall_filename)
write_locally(room_data,  room_filename)

####################
## YOUR CODE HERE ##
####################
